In [1]:
import tensorflow as tf

print(tf.__version__)



1.14.1-dev20190307


In [3]:
import pandas as pd

data = pd.read_csv('data/creditcard.csv.zip', compression='zip', skiprows=[0], header=None)

In [4]:
features = data.iloc[:, 1:30]
labels = data.iloc[:, -1]

In [5]:
print(data[:3])
print(features[:3])
print(labels[:3])

    0         1         2         3         4         5         6         7   \
0  0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1  0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2  1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   

         8         9   ...        21        22        23        24        25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   

         26        27        28      29  30  
0 -0.189115  0.133558 -0.021053  149.62   0  
1  0.125895 -0.008983  0.014724    2.69   0  
2 -0.139097 -0.055353 -0.059752  378.66   0  

[3 rows x 31 columns]
         1         2         3         4         5         6         7   \
0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(features, labels, test_size=0.2, shuffle=False, random_state=42)

In [7]:
n_features = 29

xplaceholder= tf.placeholder('float',[None,n_features])
yplaceholder = tf.placeholder('float')

# for TF 2.0
#xplaceholder= tf.Variable(n_features)
#yplaceholder = tf.Variable('float')




In [ ]:
from tensorflow.contrib import rnn
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

epochs = 8
n_classes = 1
n_units = 200
batch_size = 35

def recurrent_neural_network_model():
    layer ={ 'weights': tf.Variable(tf.random_normal([n_units, n_classes])),'bias': tf.Variable(tf.random_normal([n_classes]))}

    x = tf.split(xplaceholder, n_features, 1)
    print(x)

    lstm_cell = rnn.BasicLSTMCell(n_units)
    
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
   
    output = tf.matmul(outputs[-1], layer['weights']) + layer['bias']

    return output

def train_neural_network():
    logit = recurrent_neural_network_model()
    logit = tf.reshape(logit, [-1])

    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=yplaceholder))
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    with tf.Session() as sess:

        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        for epoch in range(epochs):
            epoch_loss = 0

            i = 0
            for i in range(int(len(X_train) / batch_size)):

                start = i
                end = i + batch_size

                batch_x = np.array(X_train[start:end])
                batch_y = np.array(y_train[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict={xplaceholder: batch_x, yplaceholder: batch_y})
                epoch_loss += c
                i += batch_size

            print('Epoch', epoch, 'completed out of', epochs, 'loss:', epoch_loss)

        pred = tf.round(tf.nn.sigmoid(logit)).eval({xplaceholder: np.array(X_test), yplaceholder: np.array(y_test)})
        f1 = f1_score(np.array(y_test), pred, average='macro')
        accuracy=accuracy_score(np.array(y_test), pred)
        recall = recall_score(y_true=np.array(y_test), y_pred= pred)
        precision = precision_score(y_true=np.array(y_test), y_pred=pred)
        print("F1 Score:", f1)
        print("Accuracy Score:",accuracy)
        print("Recall:", recall)
        print("Precision:", precision)

#clear you computational graph
#tf.reset_default_graph()
train_neural_network()




